In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import multiprocessing
import multiprocess
import feedparser as fp
import newspaper
from newspaper import news_pool
import helpers
from newspaper import Source, Article
import pandas as pd

## 1.
This is downloading newspapers as one would conventionally using the $newspaper3k$ package 

In [2]:
%%time
# Do conventional newspaper3k build...
cnn_3k_paper = newspaper.build('https://www.cnn.com/', memoize_articles=False)
rt_3k_paper = newspaper.build('https://www.rt.com/', memoize_articles=False)
bbc_3k_paper = newspaper.build('https://www.bbc.com/news', memoize_articles=False)
fox_3k_paper = newspaper.build('https://www.foxnews.com/', memoize_articles=False)
msnbc_3k_paper = newspaper.build('https://www.msnbc.com/', memoize_articles=False)

CPU times: user 6.94 s, sys: 840 ms, total: 7.78 s
Wall time: 25.3 s


In [3]:
%%time
# ... and multithread download:
papers = [cnn_3k_paper, 
          rt_3k_paper,
          bbc_3k_paper,
          fox_3k_paper,
          msnbc_3k_paper]
news_pool.set(papers, threads_per_source=2) # (5*2) = 10 threads total
news_pool.join()

CPU times: user 1min 4s, sys: 8.66 s, total: 1min 13s
Wall time: 9min 20s


In [4]:
%%time
# Process the downloaded texts:
for paper in papers:
    paper.parse_articles()

CPU times: user 8min 11s, sys: 18.2 s, total: 8min 29s
Wall time: 9min 10s


In [5]:
filepath = 'data_extra/conventional_data_pulls/'

In [6]:
%%time
for paper in papers:
    helpers.write_out_articles(newspaper_obj=paper,
                               filepath=filepath, 
                               writeout_type='conventional')

CPU times: user 29.4 s, sys: 5.8 s, total: 35.3 s
Wall time: 1min


Check for files:

In [7]:
df = pd.read_csv(filepath + '2019_3_17_bbc.csv')

In [8]:
df.text

0      Image copyright Reuters\n\nFlight data from th...
1      Image copyright Reuters Image caption Some fam...
2      Image copyright Hassan Katende / Facebook / Ha...
3      Image copyright Boeing Image caption The 737 M...
4      Media playback is unsupported on your device M...
5      Image copyright EPA Image caption Michael Ben-...
6      Image copyright Getty Images Image caption An ...
7      Image copyright Reuters Image caption Proteste...
8      Image copyright AFP Image caption Malawi has a...
9      Image copyright Sinn Féin Image caption Mary L...
10     Image copyright FIFA via Getty Images Image ca...
11     Media playback is unsupported on your device M...
12     Media playback is unsupported on your device M...
13     Image copyright AFP Image caption A tribute in...
14     Image copyright EPA, Getty Images, Reuters\n\n...
15     Image copyright Getty Images\n\nThree months a...
16     Image copyright Getty Images\n\nThe US homelan...
17     Image copyright AFP Imag